# Enable Application Gateway Ingress Controller add-on using Azure CLI
You can use Azure CLI to enable the Application Gateway Ingress Controller (AGIC) add-on for an existing Azure Kubernetes Services (AKS) cluster.  The Application Gateway Ingress Controller (AGIC) is a Kubernetes application, which makes it possible for Azure Kubernetes Service (AKS) customers to leverage Azure's native Application Gateway L7 load-balancer to expose cloud software to the Internet. AGIC monitors the Kubernetes cluster it is hosted on and continuously updates an Application Gateway, so that selected services are exposed to the Internet.  

Use AGIC add-on to expose a Kubernetes application in an existing AKS cluster through an existing Application Gateway deployed in separate virtual networks. Start by creating an AKS cluster in one virtual network and an Application Gateway in a separate virtual network to simulate existing resources. Then enable the AGIC add-on, peer the two virtual networks together, and deploy a sample application that will be exposed through the Application Gateway using the AGIC add-on. If enabling the AGIC add-on for an existing Application Gateway and existing AKS cluster in the same virtual network, then skip the peering step. The add-on prov and also offers a fully managed experience.

## Prerequisites
* [install](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli) Azure CLI
    * If you're using a local installation, sign in to the Azure CLI by using the az login command. To finish the authentication process, follow the steps displayed in your terminal. 
    * When you're prompted, install Azure CLI extensions on first use.
    * To find version or to upgrade, replace the subscription with a valid one:

In [ ]:
# Find the version
az version [--subscription] 
# Upgrade the current version
az upgrade [--all {false, true}]
           [--subscription]
           [--yes]

## Create a resource group

Create a kubernetes resource group in the location eastus.  First login:

In [ ]:
az login

After logging in, the CLI will list all available subscriptions for your account. Select your subscription using the following command:

In [ ]:
az account set --subscription <subscription id>

Then create the resource group:

In [ ]:
$ az group create --name myResourceGroup --location eastus
{
  "id": "/subscriptions/49219efc-701f-4c7e-a2ac-c600308a69e3/resourceGroups/myResourceGroup",
  "location": "eastus",
  "managedBy": null,
  "name": "myResourceGroup",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"

## Deploy a new AKS cluster
Deploy a new AKS cluster, to simulate having an existing AKS cluster that you want to enable the AGIC add-on for.

In the following example, deploy a new AKS cluster named myCluster using Azure CNI and Managed Identities in the resource group you created, myResourceGroup.

In [ ]:
az aks create -n myCluster -g myResourceGroup --network-plugin azure --enable-managed-identity 

## Deploy a new Application Gateway

Deploy a new Application Gateway, to simulate having an existing Application Gateway that you want to use to load balance traffic to your AKS cluster, myCluster. The name of the Application Gateway will be *myApplicationGateway*, but you will need to first create a public IP resource, named myPublicIp, and a new virtual network called myVnet with address space 11.0.0.0/8, and a subnet with address space 11.1.0.0/16 called mySubnet, and deploy your Application Gateway in mySubnet using myPublicIp.

When using an AKS cluster and Application Gateway in separate virtual networks, the address spaces of the two virtual networks must not overlap. The default address space that an AKS cluster deploys in is 10.0.0.0/8, so we set the Application Gateway virtual network address prefix to 11.0.0.0/8.

In [ ]:
az network public-ip create -n myPublicIp -g MyResourceGroup --allocation-method Static --sku Standard
az network vnet create -n myVnet -g myResourceGroup --address-prefix 11.0.0.0/8 --subnet-name mySubnet --subnet-prefix 11.1.0.0/16 
az network application-gateway create -n myApplicationGateway -l eastus -g myResourceGroup --sku Standard_v2 --public-ip-address myPublicIp --vnet-name myVnet --subnet mySubnet

## Enable the AGIC add-on in existing AKS cluster through Azure CLI
enable the AGIC add-on in the AKS cluster created, myCluster, and specify the AGIC add-on to use the existing Application Gateway created, myApplicationGateway.

In [ ]:
appgwId=$(az network application-gateway show -n myApplicationGateway -g myResourceGroup -o tsv --query "id") 
az aks enable-addons -n myCluster -g myResourceGroup -a ingress-appgw --appgw-id $appgwId

## Peer the two virtual networks together
Since the AKS cluster is deployed in its own virtual network and the Application Gateway in another virtual network, peer the two virtual networks together for traffic to flow from the Application Gateway to the pods in the cluster. Peering the two virtual networks requires running the Azure CLI command two separate times, to ensure that the connection is bi-directional. The first command will create a peering connection from the Application Gateway virtual network to the AKS virtual network; the second command will create a peering connection in the other direction.

In [ ]:
nodeResourceGroup=$(az aks show -n myCluster -g myResourceGroup -o tsv --query "nodeResourceGroup")
aksVnetName=$(az network vnet list -g $nodeResourceGroup -o tsv --query "[0].name")

$ aksVnetId=$(az network vnet show -n $aksVnetName -g $nodeResourceGroup -o tsv --query "id")
az network vnet peering create -n AppGWtoAKSVnetPeering -g myResourceGroup --vnet-name myVnet --remote-vnet $aksVnetId --allow-vnet-access

 "allowForwardedTraffic": false,
  "allowGatewayTransit": false,
  "allowVirtualNetworkAccess": true,
  "doNotVerifyRemoteGateways": false,
  "etag": "W/\"4fd4c1d8-a842-4c06-8c4b-45b090f7e074\"",
  "id": "/subscriptions/49219efc-701f-4c7e-a2ac-c600308a69e3/resourceGroups/myResourceGroup/providers/Microsoft.Network/virtualNetworks/myVnet/virtualNetworkPeerings/AppGWtoAKSVnetPeering",
  "name": "AppGWtoAKSVnetPeering",
  "peeringState": "Initiated",
  "peeringSyncLevel": "RemoteNotInSync",
  "provisioningState": "Succeeded",
  "remoteAddressSpace": {
    "addressPrefixes": [
      "10.0.0.0/8"
    ]
  },
  "remoteBgpCommunities": null,
  "remoteVirtualNetwork": {
    "id": "/subscriptions/49219efc-701f-4c7e-a2ac-c600308a69e3/resourceGroups/MC_myResourceGroup_myCluster_eastus/providers/Microsoft.Network/virtualNetworks/aks-vnet-19228531",
    "resourceGroup": "MC_myResourceGroup_myCluster_eastus"
  },
  "remoteVirtualNetworkAddressSpace": {
    "addressPrefixes": [
      "10.0.0.0/8"
    ]
  },
  "resourceGroup": "myResourceGroup",
  "resourceGuid": "2fd4805f-70f1-09ee-296b-cf3f3f427ae6",
  "type": "Microsoft.Network/virtualNetworks/virtualNetworkPeerings",
  "useRemoteGateways": false

$ appGWVnetId=$(az network vnet show -n myVnet -g myResourceGroup -o tsv --query "id")
az network vnet peering create -n AKStoAppGWVnetPeering -g $nodeResourceGroup --vnet-name $aksVnetName --remote-vnet $appGWVnetId --allow-vnet-access
{
  "allowForwardedTraffic": false,
  "allowGatewayTransit": false,
  "allowVirtualNetworkAccess": true,
  "doNotVerifyRemoteGateways": false,
  "etag": "W/\"f46bda3c-946f-4a9b-8351-1c7b7e3b64e4\"",
  "id": "/subscriptions/49219efc-701f-4c7e-a2ac-c600308a69e3/resourceGroups/MC_myResourceGroup_myCluster_eastus/providers/Microsoft.Network/virtualNetworks/aks-vnet-19228531/virtualNetworkPeerings/AKStoAppGWVnetPeering",
  "name": "AKStoAppGWVnetPeering",
  "peeringState": "Connected",
  "peeringSyncLevel": "FullyInSync",
  "provisioningState": "Succeeded",
  "remoteAddressSpace": {
    "addressPrefixes": [
      "11.0.0.0/8"
    ]
  },
  "remoteBgpCommunities": null,
  "remoteVirtualNetwork": {
    "id": "/subscriptions/49219efc-701f-4c7e-a2ac-c600308a69e3/resourceGroups/myResourceGroup/providers/Microsoft.Network/virtualNetworks/myVnet",
    "resourceGroup": "myResourceGroup"
  },
  "remoteVirtualNetworkAddressSpace": {
    "addressPrefixes": [
      "11.0.0.0/8"
    ]
  },
  "resourceGroup": "MC_myResourceGroup_myCluster_eastus",
  "resourceGuid": "2fd4805f-70f1-09ee-296b-cf3f3f427ae6",
  "type": "Microsoft.Network/virtualNetworks/virtualNetworkPeerings",
  "useRemoteGateways": false
}


## Deploy a sample application using AGIC

You'll now deploy a sample application to the AKS cluster you created that will use the AGIC add-on for Ingress and connect the Application Gateway to the AKS cluster. First, you'll get credentials to the AKS cluster you deployed by running the az aks get-credentials command.

In [ ]:
$ az aks get-credentials -n myCluster -g myResourceGroup
Merged "myCluster" as current context in /Users/amyma/.kube/config

Once you have the credentials to the cluster you created, run the following command to set up a sample application that uses AGIC for Ingress to the cluster. AGIC will update the Application Gateway you set up earlier with corresponding routing rules to the new sample application you deployed.

In [ ]:
$ kubectl apply -f https://raw.githubusercontent.com/Azure/application-gateway-kubernetes-ingress/master/docs/examples/aspnetapp.yaml 
pod/aspnetapp created
service/aspnetapp created
ingress.extensions/aspnetapp created

## Expose a service
Now expose a service on the same host using different routes, this app is a server for eclwatch.  Open a file, named eclwatch-ingress:

In [ ]:
nano eclwatch-ingress.yaml

In [ ]:
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: eclwatch-ingress
  annotations:
    kubernetes.io/ingress.class: nginx
spec:
  rules:
  -  http:
      paths:
      - path: /
        pathType: Prefix
        backend:
         service:
           name: eclwatch
           port: 
             number: 8010


Save this file, and apply it using:

In [ ]:
$ kubectl apply -f eclwatch-ingress.yaml
ingress.networking.k8s.io/eclwatch-ingress created

## Check that the application is reachable

In [2]:
kubectl get ingress

NAME               CLASS    HOSTS   ADDRESS   PORTS   AGE
aspnetapp          <none>   *                 80      7h36m
eclwatch-ingress   <none>   *                 80      16m


Check that the sample application you created is up and running by either visiting the IP address of the Application Gateway that you got from running the above command or check with curl. It may take Application Gateway a minute to get the update, so if the Application Gateway is still in an "Updating" state on Portal, then let it finish before trying to reach the IP address.


# Clean up resources
When no longer needed, remove the resource group, application gateway, and all related resources.


In [ ]:
az group delete --name myResourceGroup 